In [11]:
HOST = 'http://localhost:8123'
import requests
import pandas as pd
import StringIO
import datetime

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 150)

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    query = query 
    r = requests.post(host, params = {'query': query}, timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

In [63]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import StringIO
import pandas as pd

print __version__ # need 1.9.0 or greater

init_notebook_mode(connected = True)

def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    
    # plotly.offline.plot(fig, filename=filename, show_link = False)
    
    iplot(fig, show_link = False)

1.12.9


In [130]:
start_date = datetime.datetime(2016, 11, 7)
end_date = datetime.datetime(2016, 11, 25)

In [131]:
print get_clickhouse_data("DESCRIBE visits_all")

Bounce	UInt8		
Browser	String		
ClientID	UInt64		
CounterID	UInt32		
Date	Date		
DateTime	DateTime		
DeviceCategory	String		
GoalsID	Array(UInt32)		
IsNewUser	UInt8		
LastAdvEngine	String		
LastSearchEngineRoot	String		
LastSocialNetwork	String		
LastTrafficSource	String		
MobilePhone	String		
OperatingSystemRoot	String		
PageViews	Int32		
Params	Array(String)		
RegionCity	String		
RegionCountry	String		
StartURL	String		
UTMMedium	String		
VisitDuration	UInt32		
VisitID	UInt64		



In [132]:
q = 'DROP TABLE retention_users'
get_clickhouse_data(q)

u''

In [133]:
q = '''
    CREATE TABLE retention_users ENGINE = Log AS
        SELECT DISTINCT ClientID as client_id, toMonday(Date) as date
        FROM visits_all
        WHERE (Date >= '{start_date}') AND (client_id != 0)
'''.format(
    start_date = start_date.strftime('%Y-%m-%d'),
    end_date = end_date.strftime('%Y-%m-%d')
)

get_clickhouse_data(q)


u''

In [134]:
q = '''SELECT count(), uniq(client_id) FROM retention_users FORMAT TabSeparatedWithNames'''
get_clickhouse_df(q)

,count(),uniq(client_id)
0,147101,122206


In [135]:
q = '''
SELECT 
    client_id,
    min_date, 
    max_date,
    date
FROM
    (
        SELECT
            client_id,
            min(date) as min_date,
            max(date) as max_date
        FROM retention_users
        GROUP BY client_id
        HAVING min_date <= '{end_date}'
    )
    ALL INNER JOIN
    (
        SELECT 
            client_id,
            date
        FROM retention_users
    ) 
    USING client_id
LIMIT 10
FORMAT TabSeparatedWithNames
'''.format(end_date = end_date.strftime('%Y-%m-%d'))

get_clickhouse_df(q)

,client_id,min_date,max_date,date
0,147499413523122789,2016-11-21,2016-11-21,2016-11-21
1,1479648866412244456,2016-11-21,2016-11-21,2016-11-21
2,1479724114124140389,2016-11-21,2016-11-21,2016-11-21
3,1478813768357570873,2016-11-07,2016-11-07,2016-11-07
4,1475653197470988617,2016-11-21,2016-11-21,2016-11-21
5,1474481975917622301,2016-11-21,2016-11-21,2016-11-21
6,1479302399400133034,2016-11-14,2016-11-14,2016-11-14
7,1472045255713512694,2016-11-21,2016-12-19,2016-12-05
8,1472045255713512694,2016-11-21,2016-12-19,2016-11-21
9,1472045255713512694,2016-11-21,2016-12-19,2016-12-19


## Простой retention

In [136]:
q = '''
SELECT 
    uniq(client_id) as clients,
    min_date, 
    (date - min_date)/7 as week_num
FROM
    (
        SELECT
            client_id,
            min(date) as min_date,
            max(date) as max_date
        FROM retention_users
        GROUP BY client_id
        HAVING min_date <= '{end_date}'
    )
    ALL INNER JOIN
    (
        SELECT 
            client_id,
            date
        FROM retention_users
    ) 
    USING client_id
GROUP BY
    week_num,
    min_date
FORMAT TabSeparatedWithNames
'''.format(end_date = end_date.strftime('%Y-%m-%d'))

raw_ret_df = get_clickhouse_df(q)

In [137]:
ret_df = raw_ret_df.pivot_table(index = 'min_date', values = 'clients', columns = 'week_num').fillna(0).T

In [138]:
ret_df_norm = ret_df.apply(lambda x: 100*x/ret_df.loc[0], axis = 1).applymap(lambda x: x if x!=0 else None)

In [139]:
plotly_df(ret_df_norm)

## Rolling retention

In [169]:
q = '''
SELECT
    uniq(client_id) as clients,
    min_date,
    week_num
FROM
    (SELECT 
        client_id,
        min_date, 
        arrayJoin(range(toUInt64((max_date - min_date)/7) + 1)) as week_num
    FROM
        (
            SELECT
                client_id,
                min(date) as min_date,
                max(date) as max_date
            FROM retention_users
            GROUP BY client_id
            HAVING min_date <= '{end_date}'
        ))
GROUP BY
    min_date,
    week_num
FORMAT TabSeparatedWithNames
'''.format(end_date = end_date.strftime('%Y-%m-%d'))

raw_roll_ret_df = get_clickhouse_df(q)

In [170]:
roll_ret_df = raw_roll_ret_df.pivot_table(index = 'min_date', 
                                          values = 'clients', 
                                          columns = 'week_num').fillna(0).T

In [171]:
roll_ret_df_norm = roll_ret_df.apply(lambda x: 100*x/roll_ret_df.loc[0], axis = 1).applymap(lambda x: x if x!=0 else None)

In [172]:
plotly_df(roll_ret_df_norm)